In [7]:
import pandas as pd
import numpy as np
import sklearn.metrics as metrics
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, plot_roc_curve
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,roc_auc_score,classification_report,roc_curve,auc, f1_score

import pickle
import sklearn.metrics as metrics
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import plotly.graph_objects as go

from sklearn.metrics import classification_report, confusion_matrix, plot_roc_curve
from sklearn.metrics import classification_report

In [3]:
full_test = pd.read_csv("../../data/split-data/without-mhc/test/full_test.csv")
# full_test 

In [4]:
def split_dataframe_by_position(df, splits):
    dataframes = []
    index_to_split = len(df) // splits
    start = 0
    end = index_to_split
    for split in range(splits):
        temporary_df = df.iloc[start:end, :]
        dataframes.append(temporary_df)
        start += index_to_split
        end += index_to_split
    return dataframes

sp_data = split_dataframe_by_position(full_test, 11)

In [ ]:
def data_representation(X_data):
    seq_cdr3_lst = X_data['CDR3b'].to_list()
    seq_epitope_lst = X_data['epitope'].to_list()
    
    lst1 = []
    for seq in seq_cdr3_lst:
        e=one_hot_encode(seq)
        m_cdr3 = np.reshape(e, (-1, 20))
        padding = 19 - len(m_cdr3)
        c = np.concatenate((m_cdr3, np.zeros(padding * 20).reshape(padding, 20)), axis=0)
        lst1.append(c)

    lst2 = []
    for seq in seq_epitope_lst:
        e=one_hot_encode(seq)
        m_epitope = np.reshape(e, (-1, 20))
        padding = 11 - len(m_epitope)
        c = np.concatenate((m_epitope, np.zeros(padding * 20).reshape(padding, 20)), axis=0)
        lst2.append(c)
        
    res1 = list(map(lambda x: lst1[x].flatten(), range(len(lst1)))) 
    res2 = list(map(lambda x: lst2[x].flatten(), range(len(lst2))))
    
    df_res1 = pd.DataFrame(res1)
    df_res2 = pd.DataFrame(res2)
    result = pd.concat([df_res1, df_res2], axis=1)
    
    return result

In [ ]:
test01 = sp_data[0]
test02 = sp_data[1]
test03 = sp_data[2]
test04 = sp_data[3]
test05 = sp_data[4]
test06 = sp_data[5]
test07 = sp_data[6]
test08 = sp_data[7]
test09 = sp_data[8]
test10 = sp_data[9]
test11 = sp_data[10]
test12 = full_test.loc[2933711:2933721, :]

In [ ]:
py_test01 = test01.iloc[:, lambda test01: [2]]
py_test02 = test02.iloc[:, lambda test02: [2]]
py_test03 = test03.iloc[:, lambda test03: [2]]
py_test04 = test04.iloc[:, lambda test04: [2]]
py_test05 = test05.iloc[:, lambda test05: [2]]
py_test06 = test06.iloc[:, lambda test06: [2]]
py_test07 = test07.iloc[:, lambda test07: [2]]
py_test08 = test08.iloc[:, lambda test08: [2]]
py_test09 = test09.iloc[:, lambda test09: [2]]
py_test10 = test10.iloc[:, lambda test10: [2]]
py_test11 = test11.iloc[:, lambda test11: [2]]
py_test12 = test12.iloc[:, lambda test12: [2]]

In [ ]:
pX_test01 = data_representation(test01)
pX_test02 = data_representation(test02)
pX_test03 = data_representation(test03)
pX_test04 = data_representation(test04)
pX_test05 = data_representation(test05)
pX_test06 = data_representation(test06)
pX_test07 = data_representation(test07)
pX_test08 = data_representation(test08)
pX_test09 = data_representation(test09)
pX_test10 = data_representation(test10)
pX_test11 = data_representation(test11)
pX_test12 = data_representation(test12)

In [ ]:
pX_test = pd.concat([pX_test01, pX_test02, pX_test03, pX_test04, pX_test05, 
                     pX_test06, pX_test07, pX_test08, pX_test09, pX_test10, 
                     pX_test11, pX_test12], axis = 0)

pX_test = pX_test.reset_index(drop=True)

py_test = pd.concat([py_test01, py_test02, py_test03, py_test04, py_test05, 
                     py_test06, py_test07, py_test08, py_test09, py_test10, 
                     py_test11, py_test12])

In [6]:
pX_res = pd.read_csv("../../data/convert-data/without-mhc/res/X.csv")
py_res = pd.read_csv("../../data/convert-data/without-mhc/res/y.csv")

In [ ]:
lst_models = [ ('Random Forest', RandomForestClassifier(bootstrap=False, max_depth=90, max_features=5,
                         n_estimators=400, n_jobs=-1, random_state=42))]

class epitcrModel:
    def __init__(self, pmodel, pX, py):
        self.model = pmodel
        self.model.fit(pX, py)
    
    def predict(self, pnew_data):
        yhat_class = self.model.predict(pnew_data)
        return yhat_class 

    def predict_proba(self, pnew_data):
        yhat_class = self.model.predict_proba(pnew_data)
        return yhat_class 

def saveByPickle(object, path):
    pickle.dump(object, open(path, "wb"))
    print(f"{object} has been saved at {path}.")

rf_tcr = lst_models[0][1]
model_rf = epitcrModel(rf_tcr, pX_res, np.ravel(py_res))

y_rf_test_proba = model_rf.predict_proba(pX_test)
df_test = pd.DataFrame(data = y_rf_test_proba, columns = ["tmp", "predict_proba"])

In [ ]:
df_test = df_test.iloc[:, 1:]
df_prob_test = pd.concat([full_test, df_test], axis=1)
df_prob_test['binder_pred'] = np.where(df_prob_test['predict_proba'] >= 0.5, 1, 0)
df_prob_test.to_csv("../../data/predict-data/full-testset/without-mhc.csv", index=False)